# Usage

retrive word emb from large word emb

```
python sys.argv[0] --fword wordlist --femb word_emb_file --fout reduced_emb_file
```

## require

fire, tqdm

## wordlist

relatively small

each word perline

## word_emb_file

very large

optional `wordnum:num emb_dim:num` for the first line, Tencent for example. glove doesnt have this line

`word:num dim0 dim1 [<dim2, >]` for rest

read until empty line

In [1]:
import sys
if __name__ == '__main__': sys.path.append('..')
from reduce_emb.common import export_notebook

In [2]:
#export
from xs_lib.common import tqdm
import xs_lib

In [3]:
#export
class ReadlineIter:
    def __init__(self, fp, _len=None, raise_on_unmatch_len=False):
        self.fp = fp
        self._len = _len
        self.cnt = 0
        self.raise_on_unmatch_len = raise_on_unmatch_len
        if _len is not None: assert isinstance(_len, int), f"len '{_len}' must be int"
    def __next__(self):
        line = self.fp.readline()
        if len(line) == 0:
            if self._len is not None and self.cnt != self._len:
                print(f"WARNING: in ReadlineIter line cnt {self.cnt} dont match passed len {self._len}")
                if self.raise_on_unmatch_len:
                    assert self.cnt == self._len, f"line cnt {self.cnt} dont match passed len {self._len}"
            raise StopIteration
        self.cnt += 1
        return line
    def __iter__(self):
        return self
    def __len__(self):
        return self._len

In [4]:
# emb_file = '/home/zengjianjun/jupyter-ws/sematic/emb-open/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding.txt'

### test files

In [5]:
#test_export
test_emb_file = "./test_emb_file.txt"
with open(test_emb_file, "w") as fp:
    fp.write("""3 3
a 1.0 2.0 3.0
b 1.0 2.0 3.0
c 1.0 2.0 3.0""")
emb_file = test_emb_file

In [6]:
#test_export
emb_fp = open(emb_file, 'r')

In [7]:
#test_export
emb_fp.seek(0)
for i in tqdm(ReadlineIter(emb_fp, 4)):
    print(i, end='')

3 3
a 1.0 2.0 3.0
b 1.0 2.0 3.0
c 1.0 2.0 3.0


In [8]:
#test_export
test_fword_list='./test_fword_list.txt'
with open(test_fword_list, "w") as fp:
    fp.write("""a
c""")
fword_list = test_fword_list

### get_word_set

In [9]:
#test_export
print(f"reading {fword_list}")
word_set = set()
with open(fword_list, "r") as fp:
    readlineIter = ReadlineIter(fp)
    for word in tqdm(readlineIter):
        word = word.strip()
        word_set.add(word)

reading ./test_fword_list.txt


In [10]:
#export
def get_word_set(fword_list):
    print(f"reading {fword_list}")
    word_set = set()
    with open(fword_list, "r") as fp:
        readlineIter = ReadlineIter(fp)
        for word in tqdm(readlineIter):
            word = word.strip()
            word_set.add(word)
    return word_set

### output & emb

In [11]:
#export
def getEmbReadlineIter(emb_fp):
    embReadlineIter = None
    try:
        emb_word_cnt, dim = emb_fp.readline().split()
        emb_word_cnt = int(emb_word_cnt)
        print(f"emb_file has {emb_word_cnt} and {dim} dim according to first line")
        embReadlineIter = ReadlineIter(emb_fp, emb_word_cnt)
    except :
        emb_fp.seek(0)
        embReadlineIter = ReadlineIter(emb_fp)
    return embReadlineIter 

In [12]:
#test_export
fout = 'output.emb.txt'
fpout = open(fout, 'w')

In [13]:
emb_fp = open("../tests/Tencent_AILab_ChineseEmbedding_sample.txt", "r")
emb_word_cnt, dim = emb_fp.readline().split()
emb_word_cnt = int(emb_word_cnt)
print(f"emb_file has {emb_word_cnt} and {dim} dim according to first line")
embReadlineIter = ReadlineIter(emb_fp, emb_word_cnt)

emb_file has 8824330 and 200 dim according to first line


In [14]:
emb_fp = open("../tests/Tencent_AILab_ChineseEmbedding_sample.txt", "r")
getEmbReadlineIter(emb_fp)

emb_file has 8824330 and 200 dim according to first line


In [15]:
#test_export
emb_fp.seek(0)
embReadlineIter = None
print(f"reading emb_file:'f{emb_file}'")
try:
    emb_word_cnt, dim = emb_fp.readline().split()
    emb_word_cnt = int(emb_word_cnt)
    print(f"emb_file:'{emb_file}' has {emb_word_cnt} and {dim} dim according to first line")
    embReadlineIter = ReadlineIter(emb_fp, emb_word_cnt)
except:
    emb_fp.seek(0)
    embReadlineIter = ReadlineIter(emb_fp)
    
for line in tqdm(embReadlineIter):
    word = line[:line.index(' ')]
    if word in word_set:
        fpout.write(line)
fpout.close()

reading emb_file:'f./test_emb_file.txt'
emb_file:'./test_emb_file.txt' has 8824330 and 200 dim according to first line


In [16]:
#test_export
emb_fp = open(emb_file, "r")
embReadlineIter = getEmbReadlineIter(emb_fp)

fout = 'output.emb.txt'
fpout = open(fout, 'w')

for line in tqdm(embReadlineIter):
    word = line[:line.index(' ')]
    if word in word_set:
        fpout.write(line)
fpout.close()

emb_file has 3 and 3 dim according to first line


### main

In [27]:
#export
def main(femb, fword, fout):
    emb_fp = open(femb, "r")
    print(f"reading emb_file:'{femb}'")
    embReadlineIter = getEmbReadlineIter(emb_fp)
    word_set = get_word_set(fword)
    print(f"word_set: {len(word_set)} word_read")
    fpout = open(fout, 'w')
    cnt = 0
    for line in tqdm(embReadlineIter):
        word = line[:line.index(' ')]
        if word in word_set:
            fpout.write(line)
            cnt += 1
    print(f"{cnt} word emb found, total {len(word_set)} word")
    fpout.close()

In [28]:
#test_export
test_main_output = 'test_main_output.txt'
main(test_emb_file, test_fword_list, test_main_output)

reading emb_file:'./test_emb_file.txt'
emb_file has 3 and 3 dim according to first line
reading ./test_fword_list.txt



word_set: 2 word_read



2 word emb found, total 2 word


In [19]:
!cat {test_main_output}

a 1.0 2.0 3.0
c 1.0 2.0 3.0

### fire main

In [25]:
#export
import fire
if not xs_lib.common.IN_JUPYTER and __name__=="__main__":
    fire.Fire(main)

In [ ]:
export_notebook('reduce_emb.ipynb')

<IPython.core.display.Javascript object>

Converted reduce_emb.ipynb to /data/zengjianjun/jupyter-ws/sematic/emb-open/reduce_emb/reduce_emb/reduce_emb.py
Converted reduce_emb.ipynb to /data/zengjianjun/jupyter-ws/sematic/emb-open/reduce_emb/tests/test_reduce_emb.py


In [23]:
__name__

'__main__'